In [16]:
import sys
sys.path.append(r"D:\8th Semester\Machine learning\ML Labs")

In [17]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
#from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback
import os

In [18]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [19]:
def create_lstm():
    input_data = Input(shape=(time_steps, num_features))
    lstm_layer1 = LSTM(8, return_sequences=True)(input_data)
    lstm_layer2 = LSTM(20)(lstm_layer1)
    x = Flatten()(lstm_layer2)
    output_data = Dense(1)(x)
    model = Model(input_data, output_data)
    return model

In [20]:
model1 = create_lstm()
model1.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 24, 21)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 24, 8)               │             960 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 20)                  │           2,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,301 (12.89 KB)

 Trainable params: 3,301 (12.89 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
tensorflow.keras.utils.plot_model(model1 )

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [22]:
checkpoints = r'D:\8th Semester\Machine learning\ML Labs\lab10\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'D:\8th Semester\Machine learning\ML Labs\lab10'
#FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
#JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

In [23]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
#TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1]

In [24]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =create_lstm()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [25]:
path_dataset =r'D:\8th Semester\Machine learning\ML Labs\DATASET'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

C:\Users\Admin\anaconda3\envs\ML\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


((84907, 21), (24259, 21), (12130, 21))

In [26]:
time_steps=24
num_features=21

In [27]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.5754239559173584 sec


In [28]:
epochs = 5
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/5
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0711 - mae: 0.0711 - mape: 184.1845
Epoch 1: val_loss improved from inf to 0.02851, saving model to D:\8th Semester\Machine learning\ML Labs\lab10\\E1-cp-0001-loss0.03.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 41s 14ms/step - loss: 0.0711 - mae: 0.0711 - mape: 184.4757 - val_loss: 0.0285 - val_mae: 0.0285 - val_mape: 13.3580
Epoch 2/5
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0256 - mae: 0.0256 - mape: 1754.3483 
Epoch 2: val_loss improved from 0.02851 to 0.02473, saving model to D:\8th Semester\Machine learning\ML Labs\lab10\\E1-cp-0002-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 38s 14ms/step - loss: 0.0256 - mae: 0.0256 - mape: 1751.9978 - val_loss: 0.0247 - val_mae: 0.0247 - val_mape: 11.0687
Epoch 3/5
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0180 - mae: 0.0180 - mape: 245.5561 
Epoch 3: val_loss improved from 0.02473 to 0.01445, saving model to D:\8th Semester\Machine learning\ML Labs\lab10\\E1-cp-0003-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 40s 15ms/step - loss: 0.0180 - mae: 0.0180 - mape: 245.5478 - val_loss: 0.0144 - val_mae: 0.0144 - val_mape: 5.8028
Epoch 4/5
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0132 - mae: 0.0132 - mape: 44.3632
Epoch 4: val_loss improved from 0.01445 to 0.00982, saving model to D:\8th Semester\Machine learning\ML Labs\lab10\\E1-cp-0004-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 37s 14ms/step - loss: 0.0132 - mae: 0.0132 - mape: 44.4319 - val_loss: 0.0098 - val_mae: 0.0098 - val_mape: 4.2959
Epoch 5/5
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0108 - mae: 0.0108 - mape: 19.5590
Epoch 5: val_loss improved from 0.00982 to 0.00955, saving model to D:\8th Semester\Machine learning\ML Labs\lab10\\E1-cp-0005-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 36s 14ms/step - loss: 0.0108 - mae: 0.0108 - mape: 19.5824 - val_loss: 0.0096 - val_mae: 0.0096 - val_mape: 4.2964


In [29]:
import numpy as np
from keras.models import load_model
from keras.metrics import MeanAbsoluteError, MeanAbsolutePercentageError
from sklearn.preprocessing import MinMaxScaler  # or StandardScaler, depending on your use

# Load model with custom_objects to resolve 'mae' error
model = load_model(
    r'D:\8th Semester\Machine learning\ML Labs\lab10\E1-cp-0005-loss0.01.h5',
    custom_objects={
        'mae': MeanAbsoluteError(),
        'mape': MeanAbsolutePercentageError()
    }
)

# Predict using the model
y_pred_scaled = model.predict(test_X)

# Inverse transform to get actual predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)

# Error Metrics
MAE = np.mean(np.abs(y_pred - y_test_unscaled))
print('Mean Absolute Error (MAE):', np.round(MAE, 2))

MEDAE = np.median(np.abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE):', np.round(MEDAE, 2))

MSE = np.mean(np.square(y_pred - y_test_unscaled))
print('Mean Squared Error (MSE):', np.round(MSE, 2))

RMSE = np.sqrt(MSE)
print('Root Mean Squared Error (RMSE):', np.round(RMSE, 2))

MAPE = np.mean(np.abs((y_test_unscaled - y_pred) / y_test_unscaled)) * 100
print('Mean Absolute Percentage Error (MAPE):', np.round(MAPE, 2), '%')

MDAPE = np.median(np.abs((y_test_unscaled - y_pred) / y_test_unscaled)) * 100
print('Median Absolute Percentage Error (MDAPE):', np.round(MDAPE, 2), '%')

print('\nShapes:')
print('y_test_unscaled.shape:', y_test_unscaled.shape)
print('y_pred.shape:', y_pred.shape)


379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
Mean Absolute Error (MAE): 145.72
Median Absolute Error (MedAE): 114.72
Mean Squared Error (MSE): 36867.66
Root Mean Squared Error (RMSE): 192.01
Mean Absolute Percentage Error (MAPE): 1.0 %
Median Absolute Percentage Error (MDAPE): 0.79 %

Shapes:
y_test_unscaled.shape: (12105, 1)
y_pred.shape: (12105, 1)


In [30]:
checkpoints = r'D:\8th Semester\Machine learning\ML Labs\lab10\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'D:\8th Semester\Machine learning\ML Labs\lab10\E1-cp-0005-loss0.01.h5'
start_epoch= 6

In [38]:
import os
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanAbsoluteError
from keras.metrics import MeanAbsoluteError, MeanAbsolutePercentageError
from keras.regularizers import l2
from keras import backend as K

# Set paths
checkpoints = "best_model.h5"  # You can change this path

# Try to load an existing model
if not os.path.exists(checkpoints):
    model = None
else:
    model = checkpoints

# Construct the callback to save only the *best* model based on val_loss
EpochCheckpoint1 = ModelCheckpoint(
    filepath=checkpoints,
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

callbacks = [EpochCheckpoint1]

# Define model structure if none loaded
if model is None:
    print("[INFO] compiling new model...")
    
    time_steps = 24
    num_features = 21
    reg = 0.0005

    model = Sequential()
    model.add(LSTM(64, input_shape=(time_steps, num_features), kernel_regularizer=l2(reg)))
    model.add(Dense(1))

    opt = Adam(learning_rate=1e-3)
    model.compile(
        loss=MeanAbsoluteError(),
        optimizer=opt,
        metrics=[MeanAbsoluteError(), MeanAbsolutePercentageError()]
    )

else:
    print(f"[INFO] loading model from {model}...")
    model = load_model(
        model,
        custom_objects={
            'mae': MeanAbsoluteError(),
            'mape': MeanAbsolutePercentageError()
        }
    )

    # Update learning rate
    old_lr = float(K.get_value(model.optimizer.learning_rate))
    print("[INFO] old learning rate:", old_lr)

    K.set_value(model.optimizer.learning_rate, 1e-4)
    new_lr = float(K.get_value(model.optimizer.learning_rate))
    print("[INFO] new learning rate:", new_lr)


[INFO] compiling new model...


C:\Users\Admin\anaconda3\envs\ML\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


[INFO] old learning rate: 0.0010000000474974513
[INFO] new learning rate: 9.999999747378752e-05


In [39]:
epochs = 5
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/5


C:\Users\Admin\anaconda3\envs\ML\lib\site-packages\keras\src\optimizers\base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['sequential/lstm_8/lstm_cell/recurrent_kernel', 'sequential/lstm_8/lstm_cell/bias', 'sequential/dense_4/kernel', 'sequential/dense_4/bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3446 - mean_absolute_error: 0.3647 - mean_absolute_percentage_error: 95.8484 
Epoch 1: val_loss improved from inf to 0.37213, saving model to best_model.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.3446 - mean_absolute_error: 0.3647 - mean_absolute_percentage_error: 95.8552 - val_loss: 0.3721 - val_mean_absolute_error: 0.3233 - val_mean_absolute_percentage_error: 100.0000
Epoch 2/5
2642/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3683 - mean_absolute_error: 0.3821 - mean_absolute_percentage_error: 99.9987  
Epoch 2: val_loss improved from 0.37213 to 0.37048, saving model to best_model.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.3683 - mean_absolute_error: 0.3821 - mean_absolute_percentage_error: 99.9987 - val_loss: 0.3705 - val_mean_absolute_error: 0.3233 - val_mean_absolute_percentage_error: 100.0000
Epoch 3/5
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3688 - mean_absolute_error: 0.3828 - mean_absolute_percentage_error: 99.9997  
Epoch 3: val_loss improved from 0.37048 to 0.37001, saving model to best_model.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - loss: 0.3688 - mean_absolute_error: 0.3828 - mean_absolute_percentage_error: 99.9997 - val_loss: 0.3700 - val_mean_absolute_error: 0.3233 - val_mean_absolute_percentage_error: 100.0000
Epoch 4/5
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3689 - mean_absolute_error: 0.3822 - mean_absolute_percentage_error: 99.9997  
Epoch 4: val_loss improved from 0.37001 to 0.36978, saving model to best_model.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - loss: 0.3689 - mean_absolute_error: 0.3822 - mean_absolute_percentage_error: 99.9997 - val_loss: 0.3698 - val_mean_absolute_error: 0.3233 - val_mean_absolute_percentage_error: 100.0000
Epoch 5/5
2646/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3689 - mean_absolute_error: 0.3821 - mean_absolute_percentage_error: 99.9982  
Epoch 5: val_loss improved from 0.36978 to 0.36965, saving model to best_model.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - loss: 0.3689 - mean_absolute_error: 0.3821 - mean_absolute_percentage_error: 99.9982 - val_loss: 0.3696 - val_mean_absolute_error: 0.3233 - val_mean_absolute_percentage_error: 100.0000


In [41]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.metrics import MeanAbsoluteError as MAEMetric

# Load model with proper custom object handling
model = load_model(
    r'D:\8th Semester\Machine learning\ML Labs\lab10\E1-cp-0005-loss0.01.h5',
    custom_objects={'mae': MeanAbsoluteError(), 'MeanAbsoluteError': MAEMetric()}
)

# Predict and unscale
y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)

# MAE
MAE = np.mean(np.abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE):', round(MAE, 2))

# Median Absolute Error (MedAE)
MEDAE = np.median(np.abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE):', round(MEDAE, 2))

# Mean Squared Error (MSE)
MSE = np.mean(np.square(y_pred - y_test_unscaled))
print('Mean Squared Error (MSE):', round(MSE, 2))

# Root Mean Squared Error (RMSE)
RMSE = np.sqrt(MSE)
print('Root Mean Squared Error (RMSE):', round(RMSE, 2))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean(np.abs((y_test_unscaled - y_pred) / y_test_unscaled)) * 100
print('Mean Absolute Percentage Error (MAPE):', round(MAPE, 2), '%')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median(np.abs((y_test_unscaled - y_pred) / y_test_unscaled)) * 100
print('Median Absolute Percentage Error (MDAPE):', round(MDAPE, 2), '%')

# Shapes for debugging
print('\n\ny_test_unscaled.shape =', y_test_unscaled.shape)
print('y_pred.shape =', y_pred.shape)


379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step
Mean Absolute Error (MAE): 145.72
Median Absolute Error (MedAE): 114.72
Mean Squared Error (MSE): 36867.66
Root Mean Squared Error (RMSE): 192.01
Mean Absolute Percentage Error (MAPE): 1.0 %
Median Absolute Percentage Error (MDAPE): 0.79 %


y_test_unscaled.shape = (12105, 1)
y_pred.shape = (12105, 1)
